# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/aliciacaminero/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
#Función para scrapeo de libros

def scrape_books(min_rating, max_price):
    base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
    data_book = []

    for page in range(1,6): #Change the page, depend of your need
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los libros en la página
        libros = soup.find_all('article', class_='product_pod')

        for libro in libros:
            #Extract information from the book
            titulo = libro.h3.a['title']
            precio = libro.find('p', class_='price_color').text
            calificacion = libro.find('p', class_='star-rating')['class'][1]
            disponibilidad = libro.find('p', class_='instock availability').text.strip()

            # Obtener el enlace a la página del libro para más detalles
            enlace_libro = libro.h3.a['href']
            libro_url = f'http://books.toscrape.com/catalogue/{enlace_libro}'
            libro_detalles = requests.get(libro_url)
            soup_detalles = BeautifulSoup(libro_detalles.text, 'html.parser')

            # Obtener el UPC y la descripción
            upc = soup_detalles.find('th', text='UPC').find_next_sibling('td').text
            descripcion = soup_detalles.find('meta', attrs={'name': 'description'})['content'].strip()

            # Convertir la calificación a un valor numérico
            valor_calificacion = {
                'One': 1,
                'Two': 2,
                'Three': 3,
                'Four': 4,
                'Five': 5
            }.get(calificacion, 0)

            # Convertir el precio a un número flotante
            precio_float = float(precio.replace('£', '').replace('Â', '').strip()) #Delete money symbol

            # Filtrar según calificación mínima y precio máximo
            if valor_calificacion >= min_rating and precio_float <= max_price:
                data_book.append({
                    'UPC': upc,
                    'Título': titulo,
                    'Precio (£)': precio_float,
                    'Calificación': valor_calificacion,
                    'Género': soup_detalles.find('a', href=True).text,  # Género desde la página de detalles
                    'Disponibilidad': disponibilidad,
                    'Descripción': descripcion
                })

    df = pd.DataFrame(data_book)
    return df

# Parámetros de filtrado
min_rating = 4  # Cambia esto según el umbral deseado
max_price = 20.00     # Cambia esto según el precio máximo deseado

# Ejecutar el scraper
libros_df = scrape_books(min_rating, max_price)

# Mostrar el DataFrame resultante
libros_df

/var/folders/8k/8w_q7kd116v1zrzns3x24cv80000gn/T/ipykernel_5092/2386486305.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = soup_detalles.find('th', text='UPC').find_next_sibling('td').text


,UPC,Título,Precio (£),Calificación,Género,Disponibilidad,Descripción
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Books to Scrape,In stock,Aaron Ledbetterâs future had been planned ou...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Books to Scrape,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Books to Scrape,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Books to Scrape,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Books to Scrape,In stock,"Every summer, Rose goes with her mom and dad t..."
5,709822d0b5bcb7f4,Thirst,17.27,5,Books to Scrape,In stock,"On a searing summer Friday, Eddie Chapman has ..."
6,0fa6dceead7ce47a,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Books to Scrape,In stock,THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...
7,0e691eda369f4e09,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Books to Scrape,In stock,Just as Annie and Liam are busy making plans t...
8,889139b8e9c4cb36,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Books to Scrape,In stock,NEW HORROR SERIES FROM THE WALKING DEAD CREATO...
9,72f9d5be3472d34e,Mama Tried: Traditional Italian Cooking for th...,14.02,4,Books to Scrape,In stock,Cecilia Granata grew up cooking with her famil...
